## Web scraping utilizando o Beautiful Soup

O objetivo deste notebook é realizar a extração de dados de uma página web utilizando o pacote Beautiful Soup do Python. Se trata de uma ferramenta para analisar documentos, como HTML e XML. Ele cria uma árvore de análise para páginas analisadas que podem ser usadas para extrair dados de HTML, o que é útil para web scraping.

Em nosso exemplo, vamos aplicar essa técnica ao site da Glassdoor, para extrairmos informações de salários da profissão Cientista de dados, buscando o nome da empresa, e o salário para cada uma delas.

<center><img width="50%" src="https://images.prismic.io/oxylabs-sm/NWNiMmRiN2MtNzlkNC00OGIxLTg4NGUtZjZlMWY1ZWQ4NmMz_using-python-and-beautiful-soup-to-parse-data-intro-tutorial2x-3.png?auto=compress,format&rect=0,0,3113,1557&w=3113&h=1557&fm=webp&q=75"></center>

In [1]:
# importar os pacotes
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# requests do site, incluindo também um headers, especificando Mozilla como browser
cabecalho = {'user-agent':'Mozilla/5.0'}

response = requests.get('https://www.glassdoor.com.br/Sal%C3%A1rios/cientista-de-dados-sal%C3%A1rio-SRCH_KO0,18.htm',
                       headers = cabecalho)
# response.text

In [3]:
# vamos chamar a variável response, retornando [200] significa que deu certo nossa conexão
response

<Response [200]>

In [4]:
# vamos armazenar esses dados em uma variável chamada dados_brutos, para nao ter que fazer a requisição novamente
dados_brutos = response.text
# dados_brutos

In [5]:
# vamos estruturar esses dados, de forma mais organizada, através do Beuatiful Soup, salvando dentro de uma nova variável 'soup'
soup = BeautifulSoup(dados_brutos, 'html.parser')
# soup

In [6]:
# Se utilizarmos o método .prettify() podemos ver que os dados da variável soup são apresentados organizadamente,
# print(soup.prettify())

Depois de feita nossa requisição, armazenado os dados, e aplicado o Beautiful Soup nestes dados, precisamos entender um pouco a estrutura da página onde vamos realizar o scraping. Cada página obedece uma estrutura diferente. Portanto, esse trabalho analítico, para entender a página e suas estruturas não obedece um padrão que pode ser aplicado em todos os sites. 

Uma forma mais prática de entender em qual estrutura se encontram os dados desejados é clicar com o botão direito no texto desejado e ir em inspecionar, o que vai abrir uma nova janela com todos os códigos html da página.

Em nosso exemplo abaixo, após realizada a inspeção no html do site, verificamos que nossas informações estão dentro da tag h3, na classe que tem como atributo a expressão "salaries-list-item-0. -employer-name". 

**Importante ressaltar!** Como importamos o pacote re, de expressão regular, no início deste notebook, estamos aplicando o re.compile na expressão, e utilizando o .* Estamos fazendo isso pois a essa expressão contém um número que muda para cada empresa. Através do re.compile .* ele trará todos os elementos que possuem a expressao que inicia em "salaries-list-item, e que apresente qualquer outro caractere logo após, independente da quantidade de vezes em que ele apareceça. O mesmo procedimento foi feito para extrairmos os salários. Evidentemente, obedecendo a estrutura em que o dado está armazenado.

Procedimentos para encontrar os dados referentes ao nome das empresas.

In [7]:
lista_empresas = soup.find_all('h3', {'data-test': re.compile('salaries-list-item-.*-employer-name')})
# print(lista_empresas)

In [8]:
# A variável lista_empresas armazena todos os dados que obedecem a estrutura "re.compile('salaries-list-item-.*-employer-name'""
# Como é uma lista, podemos acessar cada um dos seus elementos e utilizar o.contents para desmembrar os dados
# Conforme documentado abaixo, não existe um padrão de armazenamento dos dados.
# No primeiro caso está no contents[1], e no segundo está no contents[2]
# Portanto, temos que encotrar um novo padrão
# print(lista_empresas[0].contents[1])
# print(lista_empresas[1].contents[0])

In [9]:
# Conforme documentado abaixo, se utilizarmos .find('a'), podemos localizar os dados ref aos nomes das empresas.
# Para exibirmos o texto, basta utilizarmos .text
print(lista_empresas[0].find('a').text)
print(lista_empresas[1].find('a').text)
print(lista_empresas[2].find('a').text)

Itaú Unibanco (Itaú BBA e Rede)
IBM
Semantix


In [10]:
# Após esse entendimento, podemos utilizar tudo isso dentro de uma estrutura for
for empresa in lista_empresas:
    nome_empresa = empresa.find('a').text
    print(nome_empresa)

Itaú Unibanco (Itaú BBA e Rede)
IBM
Semantix
Hospital Israelita Albert Einstein
Banco Bradesco
Propz
Radix Engenharia e Software
Autônomo (Brazil)
TOTVS
Stefanini
Softplan
Nubank
Grupo Globo
Globo
Ambev Tech
Ambev
Dasa
Ipiranga
Via
Aquarela Advanced Analytics


Procedimentos para encontrar os dados referentes aos salários.

In [11]:
lista_salarios = soup.find_all('div', {'data-test': re.compile('salaries-list-item-.*-salary-info')})
# print(lista_salarios)

In [12]:
# A variável lista_salarios armazena todos os dados que obedecem a estrutura "re.compile('salaries-list-item-.*-salary-info'""
# Como é uma lista, podemos acessar cada um dos seus elementos e utilizar o.contents para desmembrar os dados
# Conforme documentado abaixo, verificamos um certo padrão na estrutura dos dados, ao utilizarmos o .contents
print(lista_salarios[0].contents[0].text)
print(lista_salarios[1].contents[0].text)
print(lista_salarios[2].contents[0].text)

R$ 10.326
R$ 5.748
R$ 8.887


In [13]:
# Vamos salvar esse valor salário dentro de uma variável
# Vamos utilizar o .replace para limpar os dados extraídos
salario = lista_salarios[0].contents[0].text
salario = salario.replace('R$', '').replace('\xa0', '').replace('.','')
salario

'10326'

In [14]:
# Após esse entendimento, também podemos utilizar tudo isso dentro de uma estrutura for
for salario in lista_salarios:
    str_salario = salario.contents[0].text
    str_salario = str_salario.replace('R$', '').replace('\xa0', '').replace('.','')
    print(str_salario)

10326
5748
8887
12869
7375
7170
8139
6063
11490
7025
10566
12000
8636
10396
9740
9727
8216
6600
10483
5000


Vamos agora juntar os dois dados, referentes ao nome da empresa e salário, em um novo conjunto de dados. Para isso, vamos utilizar a função zip. Essa função ombina dados de vários iteráveis, de forma que o i-ésimo elemento da tupla contenha o i-ésimo elemento de cada um dos iteráveis recebidos por parâmetro. A iteração é interrompida quando já tiver percorrido todos os elementos do menor dos iteráveis.

Logo após essa junção utilizando o zip, vamos salvar os dados em um data frame

In [15]:
lista_todos_salarios = []

for empresa, salario in zip(lista_empresas, lista_salarios):
    nome_empresa = empresa.find('a').text
    
    str_salario = salario.contents[0].text
    str_salario = str_salario.replace('R$', '').replace('\xa0', '').replace('.','')
    
    lista_todos_salarios.append((nome_empresa, str_salario))
lista_todos_salarios

[('Itaú Unibanco (Itaú BBA e Rede)', '10326'),
 ('IBM', '5748'),
 ('Semantix', '8887'),
 ('Hospital Israelita Albert Einstein', '12869'),
 ('Banco Bradesco', '7375'),
 ('Propz', '7170'),
 ('Radix Engenharia e Software', '8139'),
 ('Autônomo (Brazil)', '6063'),
 ('TOTVS', '11490'),
 ('Stefanini', '7025'),
 ('Softplan', '10566'),
 ('Nubank', '12000'),
 ('Grupo Globo', '8636'),
 ('Globo', '10396'),
 ('Ambev Tech', '9740'),
 ('Ambev', '9727'),
 ('Dasa', '8216'),
 ('Ipiranga', '6600'),
 ('Via', '10483'),
 ('Aquarela Advanced Analytics', '5000')]

In [16]:
# Salvando os dados em um dataframe, para trabalharmos com essas informações no pandas
df_salarios = pd.DataFrame(lista_todos_salarios, columns=['empresa', 'salario'])
df_salarios

,empresa,salario
0,Itaú Unibanco (Itaú BBA e Rede),10326
1,IBM,5748
2,Semantix,8887
3,Hospital Israelita Albert Einstein,12869
4,Banco Bradesco,7375
5,Propz,7170
6,Radix Engenharia e Software,8139
7,Autônomo (Brazil),6063
8,TOTVS,11490
9,Stefanini,7025


In [17]:
# Vamos configurar os tipos de dados salvos no dataframe
# Verificamos que o tipo das informações é objeto. Vamos converter os dados de salário para float.
df_salarios.dtypes

empresa    object
salario    object
dtype: object

In [18]:
df_salarios['salario'] = df_salarios['salario'].astype(np.float32)
df_salarios.dtypes

empresa     object
salario    float32
dtype: object

In [19]:
df_salarios

,empresa,salario
0,Itaú Unibanco (Itaú BBA e Rede),10326.0
1,IBM,5748.0
2,Semantix,8887.0
3,Hospital Israelita Albert Einstein,12869.0
4,Banco Bradesco,7375.0
5,Propz,7170.0
6,Radix Engenharia e Software,8139.0
7,Autônomo (Brazil),6063.0
8,TOTVS,11490.0
9,Stefanini,7025.0


Agora, vamos criar uma função, que receberá todos os códigos documentados acima, de forma que possamos aplicar essa técnica de scraping do site glassdoor para qualquer cargo, sem tem que executar todos os comandos e procuras novamente. Evidentemente a função será válida apenas para o site da Glassdoor.

In [20]:
def busca_salarios_glassdoor(url_glassdoor):
    cabecalho = {'user-agent':'Mozilla/5.0'}
    
    resposta = requests.get(url_glassdoor, headers = cabecalho)
    dados_brutos = resposta.text
    
    soup = BeautifulSoup(dados_brutos, 'html.parser')
    
    lista_empresas = soup.find_all('h3', 
                {'data-test': re.compile('salaries-list-item-.*-employer-name')})
    
    lista_salarios = soup.find_all('div', 
                {'data-test': re.compile('salaries-list-item-.*-salary-info')})
    
    lista_todos_salarios = []

    for empresa, salario in zip(lista_empresas, lista_salarios):
        nome_empresa = empresa.find('a').text

        str_salario = salario.contents[0].text
        str_salario = str_salario.replace('R$', '').replace('\xa0', '').replace('.','')

        lista_todos_salarios.append((nome_empresa, str_salario))
    
    df_salarios = pd.DataFrame(lista_todos_salarios, columns=['empresa', 'salario'])
    df_salarios['salario'] = df_salarios['salario'].astype(np.float32)
    
    return df_salarios
    

Abaixo, ablicamos a função para outro link, que se refere a pesquisa de salários da profissão contador no Glassdoor. O resultado da função é um dataframe contendo todos os dados da página, já convertidos e organizados.

In [21]:
df_contador = busca_salarios_glassdoor('https://www.glassdoor.com.br/Sal%C3%A1rios/contador-sal%C3%A1rio-SRCH_KO0,8.htm')
df_contador

,empresa,salario
0,Autônomo (Brazil),3669.0
1,Contabilidade,4112.0
2,Petrobras Energía,16556.0
3,Divulga Vagas,4845.0
4,Novonor,11124.0
5,Contabilizei,5010.0
6,Vale,7597.0
7,Furnas,8934.0
8,Igreja Adventista do Sétimo Dia,2859.0
9,Autônomo,4955.0
